In [ ]:
!pip install pyqlearning

In [ ]:
!pip install Keras-RL

In [ ]:
!pip install gym

In [ ]:
import sys #Librería que provee acceso a funciones y objetos mantenidos por del intérprete
import gym #Librería para desarrollo de modelos de Aprendizaje Reforzado
import numpy as np 
import random #Generador de números aleatorios
from collections import defaultdict #módulo para  crear diccionarios]

In [ ]:
frio_caliente = gym.make('HotterColder-v0')

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
print(frio_caliente.observation_space)

Discrete(4)


In [ ]:
#Políticas de Aleatoriedad

for i_episode in range(5):
    estado = frio_caliente.reset()
    while True: #ciclo infinito a no ser que usemos un break, return.
        accion = frio_caliente.action_space.sample() # Genera la acción aleatoria entre 1 (Pedir carta) y 0 (Plantarse).
        estado, premio, fin, info = frio_caliente.step(accion) # Genera el estado (Sum Jugad - Sum Dealer - As), la recompensa, Done: Significa si el juego ha terminado para el esado que se imprime.
        if fin: #Solo cuando el juego ha terminado -> done = True se establece el premio. 
            print(info, accion)
            break

{'number': -809.5838453340549, 'guesses': 200, 'TimeLimit.truncated': False} [21.770662]
{'number': -712.8419965676944, 'guesses': 200, 'TimeLimit.truncated': False} [478.55984]
{'number': 198.01722473297878, 'guesses': 200, 'TimeLimit.truncated': False} [-1505.0308]
{'number': 916.0745511120679, 'guesses': 200, 'TimeLimit.truncated': False} [1080.2058]
{'number': -230.23900464503959, 'guesses': 200, 'TimeLimit.truncated': False} [-1348.6357]


In [ ]:
def PoliticaAjusteNumero(bj_frio_caliente):
    episodio = []
    estado = frio_caliente.reset() #se hace reseteo del estado
    accion = frio_caliente.action_space.sample()
    Sig_estado, premio, fin, info = frio_caliente.step(accion)
    episodio.append((estado, accion, premio))
    estado = Sig_estado
    while True:
        if ( estado == 1 ):
            accion = accion + frio_caliente.bounds*((1-premio)/2)
        elif ( estado == 3 ):
            accion = accion - frio_caliente.bounds*((1-premio)/2)

        Sig_estado, premio, fin, info = frio_caliente.step(accion) #Se evalua acción de acuerdo al estado que se tenga
        episodio.append((estado, accion, premio))
        estado = Sig_estado

        if fin:            
            break
    return episodio

In [ ]:
for i in range(5):
    print(PoliticaAjusteNumero(frio_caliente))

[(0, array([-1063.6544], dtype=float32), 0.16179024), (1, array([-225.44464], dtype=float32), 0.581111), (1, array([193.44434], dtype=float32), 0.8878369), (1, array([305.60745], dtype=float32), 0.98095864), (1, array([324.6488], dtype=float32), 0.9972285), (1, array([327.4203], dtype=float32), 0.99960786), (1, array([327.81244], dtype=float32), 0.99994457), (1, array([327.86786], dtype=float32), 0.99999225), (1, array([327.8756], dtype=float32), 0.9999987), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0), (1, array([327.87692], dtype=float32), 1.0

In [ ]:
def ModeloMC_AjusteNumero(juego, num_episodios, generar_episodio, gamma=1.0):
    # Se inicializan los diccionarios N y Q vacíos. Con "defaultdict" cuando no exita la llave que se consulta en el diccionario, se devolvera un array de ceros con dos componentes (Mismo tamaño que juego.action_space)
    R_suma = {0: {}, 1: {}, 2: {}, 3: {}}
    N = {0: {}, 1: {}, 2: {}, 3: {}}
    Q = {0: {}, 1: {}, 2: {}, 3: {}} # Diccionario para almacenar función Q 


    # Ciclo para cada juego
    for iter_episodio in range(1, num_episodios+1):    
        # Se genera una jugada utilizando la política de 80-20:
        episodio = generar_episodio(juego)
        # Se almacenan los estados, acciones y recompensas
        estados, accion, premio = zip(*episodio)
        descuentoPrem = np.array([gamma**i for i in range(len(premio)+1)]) #descuentoPrem es un array que contendra el monto en que se ira descontando el premio con base a las decisiones del modelo.
        # Acumulación en las tablas N y Q de modo que se pueda identificar la función acción Valor.
        for i, estado in enumerate(estados): #ever-visit
            accion_act = accion[i][0]
            if ( accion_act in R_suma[estado]):
              R_suma[estado][accion_act] += sum(premio[i:]*descuentoPrem[:-(1+i)])
            else: 
              R_suma[estado][accion_act] = sum(premio[i:]*descuentoPrem[:-(1+i)])
            if ( accion_act in N[estado] ):
              N[estado][accion_act] += 1.0
            else:
              N[estado][accion_act] = 1.0
            Q[estado][accion_act] = R_suma[estado][accion_act] / N[estado][accion_act] 
            #Just taking the mean of all the returns got by taking this action when we were in this state.
    return Q

In [ ]:
Q = ModeloMC_AjusteNumero(frio_caliente, 50, PoliticaAjusteNumero)

In [ ]:
print(Q)

{0: {-1423.2349: 195.8719843029976, 865.8988: 199.90332055091858, -891.12164: 199.51664263010025, 1375.2764: 198.2039059996605, 1031.8856: 198.41846792399883, -1264.0693: 199.22065851092339, -582.41956: 199.69968140125275, -223.7129: 197.77472966909409, -207.67894: 199.0133503973484, -1492.91: 197.70776116102934, -1801.7744: 198.5017735287547, -1616.1266: 197.98980995081365, 1027.0588: 198.70524129271507, 136.86093: 198.89010426402092, -1517.453: 198.76388807594776, -1095.0984: 198.99761819839478, -481.43524: 199.88782328367233, 918.9081: 198.34698423743248, -1325.6309: 198.5499021410942, -242.59352: 199.78699952363968, -999.1507: 198.33395692706108, -671.0157: 199.91111052036285, 415.74356: 199.332189142704, -1886.7964: 198.4438908365555, -1522.4895: 198.01140128076077, 1385.4567: 199.45627707242966, 898.0229: 198.85607156157494, -945.5108: 198.49978578090668, 1466.6603: 199.2058253288269, -742.3341: 199.50941443443298, -724.1425: 199.983651638031, 835.4525: 198.80500560998917, 1342.6